In [1]:
import pandas as pd
from collections import Counter

In [2]:
train = pd.read_json('../file/train.json', encoding='utf-8')

In [3]:
validation = pd.read_json('../file/val.json', encoding='utf-8')

In [ ]:
# validation.head(100).songs.apply(lambda e: len(e))

In [5]:
song_meta = pd.read_json('../file/song_meta.json', encoding='utf-8')

In [ ]:
Counter([len(_) for _ in train.tags]).most_common()

In [ ]:
tag_count = Counter([__ for _ in train.tags for __ in _])

In [ ]:
len(tag_count)

In [ ]:
len(list(filter(lambda e: e[1] > 10, tag_count.items())))

In [ ]:
tag_count.most_common(20)

## tag들로 word2vec 학습이 될까?

In [ ]:
from gensim.models import Word2Vec

In [ ]:
w2v = Word2Vec(train.tags, min_count=10, window=2)

In [ ]:
w2v.wv.most_similar(positive=['힙합'], negative=['감성'])

In [ ]:
w2v.wv.most_similar('감성')

In [ ]:
w2v.wv.most_similar('락')

In [ ]:
w2v.wv.most_similar('비')

In [ ]:
w2v.wv.most_similar('매장음악')

In [ ]:
w2v.wv['매장음악']

## playlist제목으로 word2vec이 될까?

In [6]:
from konlpy.tag import Komoran

In [7]:
pos = Komoran()
# pos = Kkma()

C:\Users\long8v\Anaconda3\envs\long36v\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [8]:
pos.pos('안녕하세요 현수씨')

[('안녕하세요', 'NNP'), ('현수', 'NNP'), ('씨', 'NNB')]

In [9]:
train['plyst_title_pos'] = train.plylst_title.apply(lambda e: [_[0] for _ in pos.pos(e)
                                                              if not _[1].startswith('J') 
                                                              and len(_[0])>1])

In [10]:
def get_pos(e):
    try:
        return [_[0] for _ in pos.pos(e) if not _[1].startswith('J') and len(_[0])>1]
    except:
        return []

In [30]:
validation['plyst_title_pos'] = validation.plylst_title.apply(lambda e: get_pos(e))

In [ ]:
w2v_song = Word2Vec(train.plyst_title_pos, min_count=10)

In [ ]:
w2v_song.wv.most_similar('노래')

In [ ]:
w2v_song.wv.most_similar(['사랑', '노래', '이별'])

In [ ]:
w2v_song.wv.most_similar('강아지')

#  장르

In [12]:
song_meta = pd.read_json('../file/song_meta.json', encoding='utf-8')

In [13]:
import json
import itertools

In [14]:
with open('../file/genre_gn_all.json', encoding='utf-8') as f:
    genre = json.load(f)

In [15]:
genre = {key: (genre[key[:-1]+'0'] if key[-2:] =='01' else value) for key, value in genre.items()}

In [16]:
song_meta['genre_list'] = song_meta.song_gn_dtl_gnr_basket

In [17]:
song_meta['song_gn_word'] = song_meta.genre_list.apply(lambda e: list(map(lambda x: genre[x], filter(lambda y: y in genre, e))))

In [18]:
song_meta['song_gn_word_split'] = song_meta.song_gn_word.apply(lambda e: list(itertools.chain(*map(lambda x: x.split('/'), e))))

In [19]:
song_meta_dropna = song_meta.dropna(subset=['song_gn_word_split'])

In [20]:
song_id_genre = dict(zip(song_meta_dropna['id'], song_meta_dropna['song_gn_word_split']))

In [21]:
train['genre'] = train.songs.apply(lambda e: Counter(itertools.chain(*list(map(lambda x: song_id_genre[x], e)))).most_common(5))
validation['genre'] = validation.songs.apply(lambda e: Counter(itertools.chain(*list(map(lambda x: song_id_genre[x], e)))).most_common(5))

In [22]:
train['genre_kor'] = train['genre'].apply(lambda e: list(list(zip(*e))[0]) if e else [])
validation['genre_kor'] = validation['genre'].apply(lambda e: list(list(zip(*e))[0]) if e else [])

In [23]:
corpus = train.tags + train.plyst_title_pos + train.genre_kor

In [ ]:
def cleansing(string):
    kor_alpha = re.compile('[ㄱ-ㅎ]')
    etc = re.compile(re.escape("',./;[]_-=()*&^%$#@!~"))
    re_list = [kor_alpha, etc]
    for pttn in re_list:
        string = pttn.sub("", string)
    string = string.lower()
    return string

In [ ]:
corpus = list(map(lambda e: [cleansing(word) for word in e if cleansing(word)], corpus))

In [29]:
validation.head(100)

,tags,id,plylst_title,songs,like_cnt,updt_date,plyst_title_pos,genre,genre_kor
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000,[],"[(애니메이션, 39), (OST, 25), (웹툰, 20), (게임, 19), (...","[애니메이션, OST, 웹툰, 게임, 만화]"
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,"[앨리스, 테이블]",[],[]
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000,[],"[(발라드, 11), ('00, 9), (R&B, 5), (Soul, 3), (OS...","[발라드, '00, R&B, Soul, OST]"
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000,[],"[('10-, 16), (발라드, 13), (포크, 6), (인디음악, 5), ('...","['10-, 발라드, 포크, 인디음악, '00]"
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000,[],"[(CCM, 8), (국내CCM, 4), (워십, 4)]","[CCM, 국내CCM, 워십]"
...,...,...,...,...,...,...,...,...,...
95,[산책],19457,,"[243342, 632748, 705767, 15408, 181397, 251375...",12,2017-02-28 11:23:25.000,[],"[(댄스, 61), (아이돌, 33), ('10-, 27), (여자 아이돌, 25)...","[댄스, 아이돌, '10-, 여자 아이돌, 남자 아이돌]"
96,"[재즈페스티벌, 애시드재즈, 힐링, 노라존스, 재즈팝]",129612,아름다운 악기들과 미친 음색들의 조합! 재즈 팝의 매력,[],28,2019-04-10 02:18:27.000,"[아름답, 악기, 미치, 음색, 조합, 재즈 팝, 매력]",[],[]
97,[],829,,"[439874, 256479, 337151, 627035, 256438, 44131...",26,2014-11-02 14:30:06.000,[],"[(재즈, 4), (발라드, 2), (블루스, 2), (애시드, 1), (퓨전, 1)]","[재즈, 발라드, 블루스, 애시드, 퓨전]"
98,[],145963,,"[314283, 29842, 372235, 248226, 377807, 508608...",0,2019-05-01 17:04:40.000,[],"[(댄스, 16), ('10-, 10), (아이돌, 9), (여자 아이돌, 8), ...","[댄스, '10-, 아이돌, 여자 아이돌, 발라드]"


In [34]:
train.to_pickle('../file/train_pos.pickle')
validation.to_pickle('../file/validation_pos.pickle')

In [ ]:
w2v_all = Word2Vec(train.tags, min_count=10, window=10, size=200)

In [ ]:
w2v.save('../file/w2v.model')

- genre를 추가해서 태그랑 같이 학습해도될듯